In [1]:
import pandas as pd
import numpy as np
from codebase.file_utils import (
    save_obj,
    load_obj,
)
from codebase.plot import plot_histogram, plot_density, plot_line, get_post_df
from codebase.ibis import post_process_sign, exp_and_normalise
import altair as alt
from codebase.classes_data import Data
from codebase.file_utils import (
    save_obj,
    load_obj,
    make_folder,
    path_backslash
)
from pdb import set_trace

alt.data_transformers.disable_max_rows()


DataTransformerRegistry.enable('default')

In [2]:
log_dir1 = 'log/20210223_185808_mcmc_1//'
exp_data1 = load_obj( 'complete_data', log_dir1)
ps_mcmc = load_obj('mcmc_post_samples', log_dir1)
ps_mcmc = post_process_sign(ps_mcmc)

In [2]:
log_dir3 = 'log/20210228_151930_rash6//'
exp_data = load_obj('data', log_dir3)
t = load_obj('t', log_dir3)
print(t)
# particles.particles[5].particles['beta']

99


In [4]:
exp_data.raw_data['alpha']

array([-0.53,  0.35, -1.4 , -1.4 , -0.96, -2.33])

## Plot weighted histograms

In [21]:
particles = load_obj('particles', log_dir3)
ps_smc2 = dict()
for name in particles.param_names:
    ps_smc2[name] = particles.extract_particles_in_numpy_array(name)


In [22]:
particles.get_acceptance_rate_for_particle_m(0)

array([0.90909091, 0.96875   , 0.96774194, 0.1       , 1.        ,
       0.96428571, 0.96296296, 1.        , 1.        , 1.        ,
       0.92      , 0.95833333, 0.95652174, 0.91304348, 1.        ,
       1.        , 0.95238095, 0.85      , 0.9       , 0.95      ,
       0.85      , 0.95      , 0.94736842, 0.89473684, 0.94736842,
       0.57894737, 0.47368421, 1.        , 0.77777778, 0.77777778,
       0.61111111, 0.70588235, 0.9375    , 1.        , 1.        ,
       1.        , 0.93333333, 0.46666667, 1.        , 1.        ,
       0.85714286, 0.92857143, 0.46153846, 1.        , 1.        ,
       0.61538462, 0.84615385, 0.83333333, 0.83333333, 1.        ,
       0.75      , 1.        , 1.        , 0.81818182, 1.        ,
       0.9       , 1.        , 1.        , 1.        , 1.        ,
       0.88888889, 0.88888889, 1.        , 1.        , 0.875     ,
       0.875     , 1.        , 1.        , 1.        , 0.85714286,
       1.        , 0.85714286, 0.85714286, 0.85714286, 1.     

In [23]:
corrs = load_obj('jitter_corrs', log_dir3)
corrs

{'beta': array([0.31022559, 0.36779258, 0.63573621, 0.29515409, 0.71172465,
        0.71856266, 0.65410398, 0.        , 0.64517697, 0.        ,
        0.5752898 , 0.6698543 , 0.        , 0.39994288, 0.        ,
        0.40508547, 0.12511574, 0.        , 0.        , 0.        ,
        0.        , 0.49579556, 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.767227  , 0.        , 0.        ,
        0.81651683, 0.82868885, 0.        , 0.        , 0.82307323,
        0.        , 0.        , 0.        , 0.        , 0.81991854,
        0.        , 0.8569049 , 0.        , 0.        , 0.        ,
        0.        , 0.82053525, 0.        , 0.        , 0.        ,
        0.88154038, 0.        , 0.        , 0.        , 0.88845658,
        0.        , 0.        , 0.        , 0.88241055, 0.        ,
        0.        , 0.        , 0.91417054, 0.        , 0.        ,
        0.67672011, 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.      

In [24]:
for name in ['alpha', 'beta']:
    samples = np.squeeze(ps_smc2[name])
    w = exp_and_normalise(particles.weights)
    print('\n\nEstimate')
    print(np.round(np.average(samples,axis=0, weights=w),2))
    # print('\nRead Data')
    # print(np.round(exp_data.raw_data[name],2))



Estimate
[-0.53  0.34 -1.45 -1.31 -0.86 -2.03]


Estimate
[0.44 0.44 0.44 0.44 0.44 0.44]


## Or load existing directory

In [25]:
ps_mcmc = post_process_sign(ps_mcmc)
ps_smc2 = post_process_sign(ps_smc2)


## Plot MCMC samples

In [26]:
param = 'beta'
df = get_post_df(ps_mcmc[param])
df = df[df.row==0]
df['source'] = 'mcmc'


df3 = get_post_df(ps_smc2[param]) 
df3= df3[df3.row==0]
df3['source'] = 'smc2'

plot_density(pd.concat([df, df3]), height=100)

alt.Chart(...)

In [27]:
param = 'alpha'
df = get_post_df(ps_mcmc[param]) 
df['source'] = 'mcmc'


df3 = get_post_df(ps_smc2[param]) 
df3['source'] = 'smc2'

plot_density(pd.concat([df, df3]), height=100)

alt.Chart(...)

## Plot quantiles

In [28]:
param = 'beta'
df = get_post_df(ps_smc2[param])
df_quant = df.groupby(['row', 'col'])[['value']].quantile(0.025).reset_index()
df_quant.rename({'value':'q1'}, axis=1, inplace=True)
df_quant2 = df.groupby(['row', 'col'])[['value']].quantile(0.975).reset_index()
df_quant2.rename({'value':'q2'}, axis=1, inplace=True)

df = df_quant.merge(df_quant2, on=['row', 'col'])

# simple quantile chart
df['source'] = 'smc2'
c1 = alt.Chart(df[df.row==0]).mark_bar(opacity=0.6).encode(
        alt.X('q1', title=None),
        alt.X2('q2', title=None),
        alt.Row('row'),
        alt.Column('col'),
        alt.Color('source')
)
c1

alt.Chart(...)

## chart overlaying quantiles with real data

In [29]:
df['index'] = 'r_' + df.row.astype(str)+'.c_'+df.col.astype(str)
df = df.loc[:,['index', 'q1', 'q2']]
df['source'] = 'smc2'
dd = pd.DataFrame(exp_data1.raw_data['beta'], columns=['data'])
dd['col'] = 0
dd['row'] = np.arange(6)
dd['index'] = 'r_' + dd.row.astype(str)+'.c_'+dd.col.astype(str)
dd = dd.loc[:,['index', 'data']]
plot_data = df.merge(dd, on=['index'])

In [30]:
plot_data = plot_data[plot_data['index']=='r_0.c_0']

c1 = alt.Chart(plot_data).mark_bar(opacity=0.6).encode(
    alt.X('q1', title=None,  scale=alt.Scale(domain=[-2,2])),
    alt.X2('q2', title=None),
    alt.Color('source'),    
)
    

c2 = alt.Chart(plot_data).mark_point(opacity=1, color='red').encode(
        alt.X('data', title=None),
)
(c1+c2).facet(
       'index',
    columns=1
    )


alt.FacetChart(...)

## chart overlaying quantiles with real data

In [31]:
param = 'alpha'
df = get_post_df(ps_smc2[param])
df_quant = df.groupby(['row', 'col'])[['value']].quantile(0.025).reset_index()
df_quant.rename({'value':'q1'}, axis=1, inplace=True)
df_quant2 = df.groupby(['row', 'col'])[['value']].quantile(0.975).reset_index()
df_quant2.rename({'value':'q2'}, axis=1, inplace=True)

df = df_quant.merge(df_quant2, on=['row', 'col'])

# simple quantile chart
df['source'] = 'smc2'
c1 = alt.Chart(df).mark_bar(opacity=0.6).encode(
        alt.X('q1', title=None),
        alt.X2('q2', title=None),
        alt.Row('row'),
        alt.Column('col'),
        alt.Color('source')
)
c1

alt.Chart(...)

In [32]:
df['index'] = 'r_' + df.row.astype(str)+'.c_'+df.col.astype(str)
df = df.loc[:,['index', 'q1', 'q2']]
df['source'] = 'smc2'
dd = pd.DataFrame(exp_data1.raw_data['alpha'], columns=['data'])
dd['col'] = np.arange(6)
dd['row'] = 0
dd['index'] = 'r_' + dd.row.astype(str)+'.c_'+dd.col.astype(str)
dd = dd.loc[:,['index', 'data']]
plot_data = df.merge(dd, on=['index'])

In [33]:
c1 = alt.Chart(plot_data).mark_bar(opacity=0.6).encode(
    alt.X('q1', title=None,  scale=alt.Scale(domain=[-3,3])),
    alt.X2('q2', title=None),
    alt.Color('source'),    
)
    

c2 = alt.Chart(plot_data).mark_point(opacity=1, color='red').encode(
        alt.X('data', title=None),
)
(c1+c2).facet(
       'index',
    columns=1
    )


alt.FacetChart(...)